In [1]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from dotenv import load_dotenv
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

load_dotenv()
openai_api_key = os.getenv("OPENAI_KEY")

client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=weaviate.config.AdditionalConfig(timeout=(60, 180)),
    headers={
        "X-OpenAI-Api-Key": openai_api_key  # Replace with your inference API key
    }
)

In [2]:
path_to_pdf = 'pdf_docs'

documents_text = []

for doc in os.listdir(path_to_pdf):

    doc_path = f'{path_to_pdf}/{doc}'
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
    docs = text_splitter.split_documents(pages)
    documents_text.append(docs)

documents_text = [item for sublist in documents_text for item in sublist]

In [3]:
documents_text

[Document(page_content='4/17/24, 8:43 AM Shipping your personal belongings back to Ireland\nhttps://www.citizensinformation.ie/en/returning-to-ireland/planning-your-journey-home/shipping-your-belongings-back-to-ireland/ 1/5Shipping your personal belongings back to Ireland\nIntroduc\x00on\nHow much does it cost to ship belongings to Ireland?\nDo I have to pay customs duty or tax?\nClaim relief from customs duty and VAT\nWhat documents do I need when shipping my items?\nHow do I bring my car to Ireland?\nShould I insure my items?\nMore informa\x00on\nIntroduc\x00on\nIf you’re returning to Ireland, you may want to move your belongings home too.\nYou can send smaller items by courier or post, but you should tell Revenue about this in advance.\nYou will need a shipping container if you are returning with bigger items, such as furniture or cars.\nThere are many interna\x00onal shipping services available. Do your research and shop around for the\ncompany that best suits your needs.\nHow much

In [4]:
client.collections.delete("citizens_info_docs") 
client.collections.create(
    "citizens_info_docs",

    properties=[  
        Property(
            name="source",
            data_type=DataType.TEXT,

        ),
        Property(
            name="page", 
            data_type=DataType.TEXT
        ),

        Property(
            name="content", 
            data_type=DataType.TEXT,
            vectorize_property_name=True,
            tokenization=Tokenization.LOWERCASE
        )
    ],

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="content", source_properties=['content','source']
        )
    ]
)

In [5]:
document_objs = list()

for d in documents_text:
    # Extract data from each document
    source = d.metadata['source']
    page = str(d.metadata['page'])  # page number to string
    content = d.page_content

    # Prepare properties for the data object
    document_objs.append({
        "source": source,
        "page": page,
        "content": content
    })

# Assuming you have a collection named 'Documents' set up in Weaviate
documents = client.collections.get("citizens_info_docs")
documents.data.insert_many(document_objs)  # Insert all documents at once

WeaviateInsertManyAllFailedError: Every object failed during insertion. Here is the set of all errors: vectorize target vector content: update vector: send POST request: Post "https://api.openai.com/v1/embeddings": dial tcp 104.18.7.192:443: i/o timeout

In [ ]:
query_return = documents.query.fetch_objects(
    include_vector=True, 
    limit=2)

In [ ]:
response = documents.query.near_text(
    query="How should one apply for Maternity Benefit and what are the deadlines?",
    limit=1
)

for o in response.objects:
    print(o.properties['content'])  # Object properties

4/12/24, 2:46 PM Pregnancy and social welfare payments
https://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 3/3If you are taking part in an employment scheme you have the full range of statutory employment
rights including maternity leave.
If you become pregnant while on an employment scheme (Community Employment, Rural Social
Scheme, Tús and Gateway schemes) you are en tled to take 26 weeks’ maternity leave including an
addi onal 16 weeks of op onal unpaid leave. If possible, you will be supported to complete your 52
weeks of paid work on the same or another project when you ﬁnish your maternity leave.
You should discuss with your sponsor the payment you qualify for while you are on maternity leave.
You may qualify for Maternity Beneﬁt if you have enough PRSI contribu ons. Otherwise, if your
spouse or partner is ge ng a social welfare payment, you may get a qualiﬁed adult increase on their
payment.
Page edited: 21 Sep

In [ ]:
# Assuming 'client' has been properly initialized and connected to Weaviate
response = documents.query.near_text(
    query="How to apply for social walfare",
    limit=2
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/16/24, 10:04 PM Claiming a social welfare payment
https://www.citizensinformation.ie/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/applying-for-a-social-welfare-p… 1/4Claiming a social welfare payment
Introduc on
Rules
How to apply
Where to apply
Further informa on
Introduc on
The Department of Social Protec on has a range of social welfare payments that provide ﬁnancial
support. To get a social welfare payment, you must apply for it. So, if you think you are en tled to a
payment you should always apply.
Your ﬁrst step before applying, is to ﬁnd out which payment you may be en tled to.
Payments are available for:
People who are unemployed
Families and children
Widows, widowers and surviving civil partners
Guardians or orphans
Older and re red people
People with a disability
Carers
If you get a social welfare payment you may also qualify for extra social welfare beneﬁts.
Household Budget Scheme
If you get certain social welfare payments, you can put a

In [ ]:
# Assuming 'client' has been properly initialized and connected to Weaviate
response = documents.query.near_text(
    query="What ways do I have to move stuff from the UK",
    limit=2
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/16/24, 10:04 PM Claiming a social welfare payment
https://www.citizensinformation.ie/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/applying-for-a-social-welfare-p… 1/4Claiming a social welfare payment
Introduc on
Rules
How to apply
Where to apply
Further informa on
Introduc on
The Department of Social Protec on has a range of social welfare payments that provide ﬁnancial
support. To get a social welfare payment, you must apply for it. So, if you think you are en tled to a
payment you should always apply.
Your ﬁrst step before applying, is to ﬁnd out which payment you may be en tled to.
Payments are available for:
People who are unemployed
Families and children
Widows, widowers and surviving civil partners
Guardians or orphans
Older and re red people
People with a disability
Carers
If you get a social welfare payment you may also qualify for extra social welfare beneﬁts.
Household Budget Scheme
If you get certain social welfare payments, you can put a